In [3]:
import numpy as np

## Binomial Option pricing for European Options

In [26]:
def binomial_tree_slow_my(S0,K,T,r,u,d,N,optype='C'):

    dt = T/N
    p = (np.exp(r*dt)-d)/(u-d)
    disc = np.exp(-r*dt)

    #compute asset prices at T
    S = np.zeros(N+1)
    for i in range(0,N+1):
        S[i] = S0 * u**(N-i) * d**i
    
    #compute call prices for all the asset prices created above at T
    C = np.zeros(N+1)
    for i in range(0,N+1):
        C[i] = max(S[i] - K, 0)
    
    for i in range(N,0,-1):
        Ct = np.zeros(i)
        for j in range(0,i):
            Ct[j] = disc*(p*C[j] + (1-p)*C[j+1])
        C = Ct
    
    return C[0]
    

In [27]:
binomial_tree_slow_my(100,100,1, 0.06, 1.1, 0.9090909090909091,3)

10.145735799928826

In [33]:
2**np.arange(1,3) + 2**np.arange(1,3)

array([4, 8], dtype=int32)

In [58]:
def binomial_tree_fast_my(S0,K,T,r,u,d,N,optype='C'):

    dt = T/N
    p = (np.exp(r*dt)-d)/(u-d)
    disc = np.exp(-r*dt)

    C = S0 * d**(np.arange(N,-1,-1)) * u**(np.arange(0,N+1,1))

    C = np.maximum(C-K, np.zeros(N+1))

    for i in np.arange(N,0,-1):
        C = disc * (p*C[1:i+1] + (1-p)*C[0:i])

    return C[0]

In [57]:
binomial_tree_fast_my(100,100,1, 0.06, 1.1, 0.9090909090909091,3)

array([10.1457358])

Binomial option pricing for an European option for an underlying that pays dividends

In [49]:
def binomial_tree_fast_div(S0,K,T,r,q,std,N,optype='C'):

    dt = T/N
    a = np.exp((r-q)*dt)
    u = np.exp(std*np.sqrt(dt))
    d = 1/u
    p = (a-d)/(u-d)
    disc = np.exp(-r*dt)

    C = S0 * d**(np.arange(N,-1,-1)) * u**(np.arange(0,N+1,1))

    C = np.maximum(C-K, np.zeros(N+1))

    for i in np.arange(N,0,-1):
        C = disc * (p*C[1:i+1] + (1-p)*C[0:i])

    return C[0]

In [51]:
binomial_tree_fast_div(100,100,1, 0.06, 0.03,0.4,3)

17.878801831184887

Barrier exotic option pricing for European options

In [43]:
def binomial_tree_barrier(S0,K,T,H,r,u,d,N,optype='C'):
    
    dt = T/N
    p = (np.exp(r*dt) - d)/ (u-d)
    disc = np.exp(-r*dt)

    S = np.zeros(N+1)
    for i in range(0,N+1):
        S[i] = S0 * u**(i) * d**(N-i)
    
    C = np.zeros(N+1)
    for i in range(0,N+1):
        if S[i] > H:
            C[i] = 0
        else:
            C[i] = max(S[i]-K, 0)
    
    for i in np.arange(N-1, -1, -1):
        for j in range(0,i+1):
            S = S0 * u**(j) * d**(i-j)
            if S > H:
                C[j] = 0
            else:
                C[j] = disc*(p*C[j+1] + (1-p)*C[j])
    return C[0]

In [44]:
binomial_tree_barrier(100,100,1, 125,0.06, 1.1, 0.9090909090909091,3)

4.00026736854323

In [45]:
def binomial_tree_barrier_fast(S0,K,T,H,r,u,d,N,optype='C'):

    dt = T/N
    p = (np.exp(r*dt) - d)/ (u-d)
    disc = np.exp(-r*dt)

    S = S0 * u**(np.arange(0,N+1,1)) * d**(np.arange(N,-1,-1))

    C = np.maximum(S-K,0)

    C[S>=H]=0

    for i in np.arange(N-1,-1,-1):
        S = S0 * u**(np.arange(0,i+1,1)) * d**(np.arange(i,-1,-1))
        C[:i+1] = disc * (p*C[1:i+2] + (1-p)*C[0:i+1])
        C = C[:-1]
        C[S>=H] =0
    return C[0]

In [46]:
binomial_tree_barrier_fast(100,100,1, 125,0.06, 1.1, 0.9090909090909091,3)

4.00026736854323

#### Greeks

In [61]:
def binomial_tree_fast_price(S0,K,T,r,u,d,N,t=0,optype='C'):

    dt = T/N
    p = (np.exp(r*dt)-d)/(u-d)
    disc = np.exp(-r*dt)

    C = S0 * d**(np.arange(N,-1,-1)) * u**(np.arange(0,N+1,1))

    C = np.maximum(C-K, np.zeros(N+1))

    for i in np.arange(N,t,-1):
        C = disc * (p*C[1:i+1] + (1-p)*C[0:i])

    return C

In [62]:
def binomial_tree_fast_div_price(S0,K,T,r,q,std,N,t=0,optype='C'):

    dt = T/N
    a = np.exp((r-q)*dt)
    u = np.exp(std*np.sqrt(dt))
    d = 1/u
    p = (a-d)/(u-d)
    disc = np.exp(-r*dt)

    C = S0 * d**(np.arange(N,-1,-1)) * u**(np.arange(0,N+1,1))

    C = np.maximum(C-K, np.zeros(N+1))

    for i in np.arange(N,t,-1):
        C = disc * (p*C[1:i+1] + (1-p)*C[0:i])

    return C

In [79]:
abc = binomial_tree_fast_div_price(100,100,1, 0.06, 0.03,0.4,3,1).tolist()
abc.sort(reverse=True)
abc

[33.09515992195429, 5.375755694817835]

In [93]:
class BinomialTree:
    def __init__(self,S0,K,T,r,q,std,N,optype='C'):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.q = q
        self.std = std
        self.N = N
        self.optype = optype

        self.dt = T/N
        self.a = np.exp((r-q)*self.dt)
        self.u = np.exp(std*np.sqrt(self.dt))
        self.d = 1/self.u
        self.p = (self.a-self.d)/(self.u-self.d)
        self.disc = np.exp(-r*self.dt)

    def price(self,t=0,std=None,r=None):
            
        std = std if std is not None else self.std
        r = r if r is not None else self.r

        if (r != None) or (std != None):
            self.dt = self.T / self.N
            self.a = np.exp((r - self.q) * self.dt)
            self.u = np.exp(std * np.sqrt(self.dt))
            self.d = 1 / self.u
            self.p = (self.a - self.d) / (self.u - self.d)
            self.disc = np.exp(-r * self.dt)

        S = self.S0 * self.d**(np.arange(self.N, -1, -1)) * self.u**(np.arange(0, self.N+1, 1))

        if self.optype == 'C':
            C = np.maximum(S-self.K, 0)
        else:
            C = np.maximum(self.K-S, 0)
        
        if t == 0:
            for i in np.arange(self.N,0,-1):
                C = self.disc * (self.p*C[1:i+1] + (1-self.p)*C[0:i])
                if i == 3:
                    self.C_2 = C
                elif i == 2:
                    self.C_1 = C
                else:
                    continue
                return C[0]
        else:
            for i in np.arange(self.N,t,-1):
                C = self.disc * (self.p*C[1:i+1] + (1-self.p)*C[0:i])
                C = C.tolist()
                C.sort(reverse=True)
                return C
                
    def delta(self,t=1):
        if t==1:
            C_t = self.C_1
        else:
            C_t = self.price(t=1)
        delta = (C_t[0] - C_t[1])/(self.S0*self.u - self.S0*self.d)
        return delta
    
    def gamma(self,t=2):
        if t==2:
            C_t = self.C_2
        else:
            C_t = self.price(t=2)
        delta_plus = (C_t[0] - C_t[1])/(self.S0*self.u**2 - self.S0)
        delta_minus = (C_t[1] - C_t[2])/(self.S0 - self.S0*self.d**2)
        h = 0.5*(self.S0*self.u**2 - self.S0*self.d**2)

        gamma = (delta_plus - delta_minus)/h

        return gamma
    
    def theta(self,t=2):
        if t==2:
            C_t = self.C_2
        else:
            C_t = self.price(t=2)

        theta = (C_t[0] - C_t[2])/(2*self.dt)

        return theta
    
    def vega(self, bp=1):
        bpp = bp/100
        C_std_plus = self.price(std=self.std + bpp)
        C_std_minus = self.price(std=self.std - bpp)

        vega = (C_std_plus - C_std_minus)/(2*bpp)

        return vega
    
    def rho(self, bp=1):
        bpp = bp/100
        C_r_plus = self.price(r=self.r + bpp)
        C_r_minus = self.price(r=self.r - bpp)

        rho = (C_r_plus - C_r_minus)/(2*bpp)

        return rho


In [119]:
import numpy as np

class BinomialTree:
    def __init__(self, S0, K, T, r, q, std, N, optype='C'):
        self.S0 = S0
        self.K = K
        self.T = T
        self.r = r
        self.q = q
        self.std = std
        self.N = N
        self.optype = optype

        self.dt = T / N
        self.a = np.exp((r - q) * self.dt)
        self.u = np.exp(std * np.sqrt(self.dt))
        self.d = 1 / self.u
        self.p = (self.a - self.d) / (self.u - self.d)
        self.disc = np.exp(-r * self.dt)

        self.C_1 = None
        self.C_2 = None

    def price(self, t=0, std=None, r=None):
        std = std if std is not None else self.std
        r = r if r is not None else self.r

        if r is not None or std is not None:
            self.dt = self.T / self.N
            self.a = np.exp((r - self.q) * self.dt)
            self.u = np.exp(std * np.sqrt(self.dt))
            self.d = 1 / self.u
            self.p = (self.a - self.d) / (self.u - self.d)
            self.disc = np.exp(-r * self.dt)

        # Calculate initial asset prices at maturity
        S = self.S0 * self.d ** (np.arange(self.N, -1, -1)) * self.u ** (np.arange(0, self.N + 1, 1))

        # Calculate option payoffs at maturity
        if self.optype == 'C':
            C = np.maximum(S - self.K, 0)
        else:
            C = np.maximum(self.K - S, 0)

        # Perform backward induction to price the option
        if t == 0:
            for i in np.arange(self.N, 0, -1):
                C = self.disc * (self.p * C[1:i+1] + (1 - self.p) * C[0:i])
                # Save intermediate option values for greeks calculation
            return C[0]  # Return option price at the root node

        else:
            for i in np.arange(self.N, t, -1):
                C = self.disc * (self.p * C[1:i+1] + (1 - self.p) * C[0:i])
            C = C.tolist()
            C.sort(reverse=True)
            return C  # Return option values at the specified time step

    def delta(self, t=1):
        C_t = self.price(t=1)
        delta = (C_t[0] - C_t[1]) / (self.S0 * self.u - self.S0 * self.d)
        return delta

    def gamma(self, t=2):

        C_t = self.price(t=2)

        delta_plus = (C_t[0] - C_t[1]) / (self.S0 * self.u ** 2 - self.S0)
        delta_minus = (C_t[1] - C_t[2]) / (self.S0 - self.S0 * self.d ** 2)
        h = 0.5 * (self.S0 * self.u ** 2 - self.S0 * self.d ** 2)

        gamma = (delta_plus - delta_minus) / h

        return gamma

    def theta(self, t=2):

        C_t = self.price(t=2)

        theta = (C_t[0] - C_t[2]) / (2 * self.dt)

        return theta

    def vega(self, bp=1):
        bpp = bp / 100
        C_std_plus = self.price(std=self.std + bpp)
        C_std_minus = self.price(std=self.std - bpp)

        vega = (C_std_plus - C_std_minus) / (2 * bpp)

        return vega

    def rho(self, bp=1):
        bpp = bp / 100
        C_r_plus = self.price(r=self.r + bpp)
        C_r_minus = self.price(r=self.r - bpp)

        rho = (C_r_plus - C_r_minus) / (2 * bpp)

        return rho


In [120]:
option = BinomialTree(S0=100, K=100, T=1, r=0.05, q=0.02, std=0.2, N=6)

In [121]:
option.delta()

0.5849662682044122

In [122]:
option.gamma()

0.022094089566772588

In [123]:
option.theta()

57.39914529083102

In [124]:
option.vega()

36.35951586286383

In [125]:
option.rho()

49.38564202421816

## Binomial Option pricing for American Options

In [36]:
def binomial_american_fast(S0,K,T,r,u,d,N,optype='C'):

    dt = T/N
    disc = np.exp(-r*dt)
    p = (np.exp(r*dt)-d)/(u-d)

    S = S0 * d**(np.arange(N,-1,-1)) * u**(np.arange(0,N+1,1))

    if optype=='C':
        C = np.maximum(S-K, 0)
    else:
        C = np.maximum(K-S, 0)

    for i in np.arange(N-1,-1,-1):
        S = S0 * d**(np.arange(i,-1,-1)) * u**(np.arange(0,i+1,1))
        C[:i+1] = (disc*(p*C[1:i+2] + (1-p)*C[0:i+1]))
        C = C[:-1]
        if optype=='C':
            C = np.maximum(S-K, C)
        else:
            C = np.maximum(K-S, C)
    
    return C[0]

In [37]:
binomial_american_fast(100,100,1, 0.06, 1.1, 0.9090909090909091,3,'P')

4.654588754602527

In [54]:
def binomial_american_fast_div(S0,K,T,r,q,std,N,optype='C'):

    dt = T/N
    a = np.exp((r-q)*dt)
    u = np.exp(std*np.sqrt(dt))
    d = 1/u
    p = (a-d)/(u-d)
    disc = np.exp(-r*dt)

    S = S0 * d**(np.arange(N,-1,-1)) * u**(np.arange(0,N+1,1))

    if optype=='C':
        C = np.maximum(S-K, 0)
    else:
        C = np.maximum(K-S, 0)

    for i in np.arange(N-1,-1,-1):
        S = S0 * d**(np.arange(i,-1,-1)) * u**(np.arange(0,i+1,1))
        C[:i+1] = (disc*(p*C[1:i+2] + (1-p)*C[0:i+1]))
        C = C[:-1]
        if optype=='C':
            C = np.maximum(S-K, C)
        else:
            C = np.maximum(K-S, C)
    
    return C[0]

In [55]:
binomial_american_fast_div(100,100,1, 0.06, 0.03,0.4,3)

17.878801831184887